# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-25.0660,19.63,60,84,4.49,PN,2024-09-14
1,1,arraial do cabo,-22.9661,-22.9661,23.36,83,0,12.57,BR,2024-09-14
2,2,karratha,-20.7377,-20.7377,27.82,33,88,8.48,AU,2024-09-14
3,3,bukama,-9.2000,-9.2000,24.92,28,13,1.00,CD,2024-09-14
4,4,papatowai,-46.5619,-46.5619,6.44,97,100,3.75,NZ,2024-09-14


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    size="Humidity",
    color="City"
    )

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[((city_data_df["Max Temp"] <= 27.0) & (city_data_df["Max Temp"] >= 20.0))\
                                 & (city_data_df["Cloudiness"] == 0) & (city_data_df["Wind Speed"] <= 4.5),:]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna(how="any")

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,9,uturoa,-16.7333,-16.7333,26.16,79,0,4.45,PF,2024-09-14
55,55,tostado,-29.2320,-29.2320,21.66,29,0,4.44,AR,2024-09-14
67,67,westport,41.1415,41.1415,24.55,69,0,1.54,US,2024-09-14
74,74,jamestown,42.0970,42.0970,24.80,53,0,2.06,US,2024-09-14
101,101,turbat,26.0023,26.0023,26.47,72,0,2.07,PK,2024-09-14
109,109,lander,42.8330,42.8330,25.45,14,0,2.57,US,2024-09-14
179,179,kharan,28.5833,28.5833,26.66,16,0,1.08,PK,2024-09-14
330,330,al bawiti,28.3492,28.3492,25.52,55,0,3.57,EG,2024-09-14
396,396,cornwall,45.0181,45.0181,24.06,69,0,0.00,CA,2024-09-14
411,411,winthrop harbor,42.4789,42.4789,26.43,66,0,3.09,US,2024-09-14


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
9,uturoa,PF,-16.7333,-16.7333,79,
55,tostado,AR,-29.2320,-29.2320,29,
67,westport,US,41.1415,41.1415,69,
74,jamestown,US,42.0970,42.0970,53,
101,turbat,PK,26.0023,26.0023,72,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "limit":20,
    "categories":"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
uturoa - nearest hotel: No hotel found
tostado - nearest hotel: No hotel found
westport - nearest hotel: Elikti Butik Otel
jamestown - nearest hotel: No hotel found
turbat - nearest hotel: No hotel found
lander - nearest hotel: No hotel found
kharan - nearest hotel: No hotel found
al bawiti - nearest hotel: No hotel found
cornwall - nearest hotel: No hotel found
winthrop harbor - nearest hotel: Hotel Chateau Chikovani
al jawf - nearest hotel: No hotel found
mount pleasant - nearest hotel: No hotel found
chibougamau - nearest hotel: No hotel found
sainte-catherine - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
9,uturoa,PF,-16.7333,-16.7333,79,No hotel found
55,tostado,AR,-29.2320,-29.2320,29,No hotel found
67,westport,US,41.1415,41.1415,69,Elikti Butik Otel
74,jamestown,US,42.0970,42.0970,53,No hotel found
101,turbat,PK,26.0023,26.0023,72,No hotel found
109,lander,US,42.8330,42.8330,14,No hotel found
179,kharan,PK,28.5833,28.5833,16,No hotel found
330,al bawiti,EG,28.3492,28.3492,55,No hotel found
396,cornwall,CA,45.0181,45.0181,69,No hotel found
411,winthrop harbor,US,42.4789,42.4789,66,Hotel Chateau Chikovani


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    size="Humidity",
    color="City",
    hover_cols=["Hotel Name","Country"]
    )

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)